In [324]:
# libraries to be imported
import pandas as pd
from datetime import datetime
import haversine as hs
from haversine import Unit

In [325]:
# Load the dataset
data = pd.read_csv('CDR_Data_Ward.csv')

# Convert 'timestamp' to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True)

# Sort data to ensure that deduplication keeps the first occurrence
data.sort_values(by=['device.id', 'KGISWardNo', 'timestamp'], inplace=True)

# Function to drop duplicates within a 10 minute window
def drop_duplicates_within_time(df, time_delta_minutes=10):
    df = df.reset_index(drop=True)
    to_keep = []
    last_time = None

    for idx, row in df.iterrows():
        current_time = row['timestamp']
        if last_time is None or (current_time - last_time).total_seconds() / 60 > time_delta_minutes:
            to_keep.append(idx)
            last_time = current_time
    
    return df.loc[to_keep]

# Apply the function to each group of 'device.id' within each 'id'
deduplicated_data = data.groupby(['KGISWardNo', 'device.id']).apply(drop_duplicates_within_time).reset_index(drop=True)
# Save the deduplicated data to a new file
len(deduplicated_data)


131542

In [326]:
device_id_counts = deduplicated_data.groupby('device.id')['KGISWardNo'].nunique()

# Get the device IDs that exist in multiple KGISWardNos
multiple_kgiswardno_device_ids = device_id_counts[device_id_counts > 1].index

# Filter the original DataFrame to keep rows where 'device.id' is in multiple_kgiswardno_device_ids
deduplicated_data = deduplicated_data[deduplicated_data['device.id'].isin(multiple_kgiswardno_device_ids)]


In [327]:
deduplicated_data

,device.id,andr.iph,first.event,last.event,tot.events,tot.event.dur.mins,lat,long,timestamp,horiz.accu,...,LGD_WardCo,KGISWardNo,KGISWardNa,KGISTownCo,created_us,created_da,last_edite,last_edi_1,SHAPE_STAr,SHAPE_STLe
0,M3UxcWlyMWdtcGhrZzoyajFzcXF2YWltdW1n,False,27-03-2023 06:26,27-03-2023 23:02,31,995.6333,13.11420,77.60643,2023-03-27 03:33:20+00:00,2.0,...,1303139,1,Kempegowda Ward,2003,SA,22-10-2021,SA,28-01-2022,10716222.45,21217.84300
1,M3UxcWlyMWdtcGhrZzoyajFzcXF2YWltdW1n,False,27-03-2023 06:26,27-03-2023 23:02,31,995.6333,13.11429,77.60647,2023-03-27 04:14:17+00:00,2.0,...,1303139,1,Kempegowda Ward,2003,SA,22-10-2021,SA,28-01-2022,10716222.45,21217.84300
2,M3UxcWlyMWdtcGhrZzoyajFzcXF2YWltdW1n,False,27-03-2023 06:26,27-03-2023 23:02,31,995.6333,13.11417,77.60641,2023-03-27 05:26:01+00:00,2.0,...,1303139,1,Kempegowda Ward,2003,SA,22-10-2021,SA,28-01-2022,10716222.45,21217.84300
3,M3UxcWlyMWdtcGhrZzoyajFzcXF2YWltdW1n,True,27-03-2023 06:26,27-03-2023 23:02,31,995.6333,13.11421,77.60648,2023-03-27 06:01:57+00:00,2.0,...,1303139,1,Kempegowda Ward,2003,SA,22-10-2021,SA,28-01-2022,10716222.45,21217.84300
4,M3UxcWlyMWdtcGhrZzoyajFzcXF2YWltdW1n,True,27-03-2023 06:26,27-03-2023 23:02,31,995.6333,13.11418,77.60644,2023-03-27 06:38:31+00:00,2.0,...,1303139,1,Kempegowda Ward,2003,SA,22-10-2021,SA,28-01-2022,10716222.45,21217.84300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131537,cnV1ZmlxN2RzYjplb3N0YmRtY2x0aHVr,False,27-03-2023 07:22,28-03-2023 03:24,34,1202.1500,12.87336,77.54184,2023-03-27 17:01:14+00:00,2.0,...,1302870,198,Hemmigepura,2003,SA,22-10-2021,SA,28-01-2022,35793986.56,51637.22214
131538,cnV1ZmlxN2RzYjplb3N0YmRtY2x0aHVr,False,27-03-2023 07:22,28-03-2023 03:24,34,1202.1500,12.87326,77.54177,2023-03-27 17:31:58+00:00,2.0,...,1302870,198,Hemmigepura,2003,SA,22-10-2021,SA,28-01-2022,35793986.56,51637.22214
131539,cnV1ZmlxN2RzYjplb3N0YmRtY2x0aHVr,False,27-03-2023 07:22,28-03-2023 03:24,34,1202.1500,12.87341,77.54168,2023-03-27 21:53:04+00:00,2.0,...,1302870,198,Hemmigepura,2003,SA,22-10-2021,SA,28-01-2022,35793986.56,51637.22214
131540,dXFlMnYyNXQ0aWw1OmJ1ZmR0cG5pbG0wN2U=,True,27-03-2023 06:45,28-03-2023 01:17,27,1111.6670,12.92477,77.49875,2023-03-27 16:09:51+00:00,1.0,...,1302870,198,Hemmigepura,2003,SA,22-10-2021,SA,28-01-2022,35793986.56,51637.22214


## Calculating and storing origin , destination and travel time , saving it into meta_travel

In [328]:
deduplicated_data.sort_values(by=['device.id','timestamp'], inplace = True)

In [329]:
def get_distance(loc1, loc2):
    return round(hs.haversine(loc1,loc2,unit=Unit.KILOMETERS),2)

def get_travel_time(origin, destination, origin_time, destination_time, show_time):
    """Calculates the travel time in minutes between two timestamps if origin and destination are the same."""
    if origin == destination and not show_time:
        return 0
    else:
        # Convert string timestamps to datetime objects
        # Calculate difference in minutes
        return (destination_time - origin_time).total_seconds() / 60

In [330]:
def store_meta_travel(data):
    temp = None
    meta = []
    meta_travel = []
    previous_ward = None
    previous_time = None
    previous_lat_long = None
    
    for index, rows in data.iterrows():
        if temp == None:
            temp = rows['device.id']
        if rows['device.id'] == temp:
            if previous_ward!= None:
                meta.append({
                    'origin': previous_ward,
                    'destination': rows['KGISWardNo'],
                    'origin_time': previous_time,
                    'destination_time': rows['timestamp'],
                    'travel_distance': get_distance(previous_lat_long, [rows['lat'], rows['long']]),
                    'travel_time' : get_travel_time(previous_ward, rows['KGISWardNo'], previous_time, rows['timestamp'], False),
                    'time_spent' : get_travel_time(previous_ward, rows['KGISWardNo'], previous_time, rows['timestamp'], True),
                    'origin_lat' : previous_lat_long[0],
                    'origin_long': previous_lat_long[1],
                    'destination_lat': rows['lat'],
                    'destination_long': rows['long']
                })
            else:
                meta.append({})
            previous_ward = rows['KGISWardNo']
            previous_time = rows['timestamp']
            previous_lat_long = [rows['lat'], rows['long']]
            temp = rows['device.id']

        else:
#             previous_ward = rows['KGISWardNo']
            previous_ward = rows['KGISWardNo']
            previous_time = rows['timestamp']
            previous_lat_long = [rows['lat'], rows['long']]
            temp = rows['device.id']
            meta.append({})
            meta_travel.append(meta)
            meta = []
            
    meta_travel.append(meta)

    return meta_travel       

In [331]:
unique_device_ids = deduplicated_data[['device.id']].drop_duplicates()
unique_device_ids['meta_travel'] = None
meta_travel = store_meta_travel(deduplicated_data)
unique_device_ids['meta_travel'] = pd.Series(meta_travel).values



In [332]:
unique_device_ids = unique_device_ids.reset_index()

## Adding this meta data into different columns , in intial data

In [333]:
def add_columns_of_meta(data, meta_data):
    # Reset the index of the DataFrame to ensure alignment
    data = data.reset_index(drop=True)
    
    i = 0  # index for `meta_travel` items
    for index, rows in meta_data.iterrows():
        for meta_travel in rows['meta_travel']:  # Assuming rows['meta_travel'] is a list of dictionaries
            if i >= len(data):
                break  # Safeguard to prevent out-of-index errors
            data.at[i, 'origin'] = meta_travel.get('origin', None)
            data.at[i, 'destination'] = meta_travel.get('destination', None)
            data.at[i, 'origin_time'] = meta_travel.get('origin_time', None)
            data.at[i, 'destination_time'] = meta_travel.get('destination_time', None)
            data.at[i, 'travel_distance'] = meta_travel.get('travel_distance', None)
            data.at[i, 'travel_time'] = meta_travel.get('travel_time', None)
            data.at[i, 'time_spent'] = meta_travel.get('time_spent', None)
            data.at[i, 'origin_lat'] = meta_travel.get('origin_lat', None)
            data.at[i, 'origin_long'] = meta_travel.get('origin_long', None)
            data.at[i, 'destination_lat'] = meta_travel.get('destination_lat', None)
            data.at[i, 'destination_long'] = meta_travel.get('destination_long', None)
            i += 1  # Move to the next row in data

    return data


In [334]:
deduplicated_data_new = deduplicated_data
deduplicated_data_new['origin'] = None
deduplicated_data_new['destination'] = None
deduplicated_data_new['origin_time'] = None
deduplicated_data_new['destination_time'] = None
deduplicated_data_new['travel_distance'] = None
deduplicated_data_new['travel_time'] = None
deduplicated_data_new['time_spent'] = None
deduplicated_data_new['origin_lat'] = None
deduplicated_data_new['origin_long'] = None
deduplicated_data_new['destination_lat'] = None
deduplicated_data_new['destination_long'] = None

deduplicated_data_new = add_columns_of_meta(deduplicated_data, unique_device_ids)

In [335]:
# deduplicated_data_new.to_csv('Travel_Data.csv')

In [336]:
deduplicated_data_new = deduplicated_data_new.dropna(subset=['origin_lat', 'origin_long', 'destination_lat', 'destination_long'])

In [337]:
deduplicated_data_new

,device.id,andr.iph,first.event,last.event,tot.events,tot.event.dur.mins,lat,long,timestamp,horiz.accu,...,destination,origin_time,destination_time,travel_distance,travel_time,time_spent,origin_lat,origin_long,destination_lat,destination_long
1,M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk,True,27-03-2023 08:46,28-03-2023 01:28,56,1001.867,13.06102,77.56255,2023-03-27 03:30:58+00:00,20.0,...,9,2023-03-27 03:16:47+00:00,2023-03-27 03:30:58+00:00,0.0,0,14.183333,13.06103,77.56252,13.06102,77.56255
2,M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk,True,27-03-2023 08:46,28-03-2023 01:28,56,1001.867,13.06064,77.56214,2023-03-27 04:07:17+00:00,19.0,...,9,2023-03-27 03:30:58+00:00,2023-03-27 04:07:17+00:00,0.06,0,36.316667,13.06102,77.56255,13.06064,77.56214
3,M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk,False,27-03-2023 08:46,28-03-2023 01:28,56,1001.867,13.00868,77.57170,2023-03-27 04:45:26+00:00,11.0,...,35,2023-03-27 04:07:17+00:00,2023-03-27 04:45:26+00:00,5.87,38.15,38.15,13.06064,77.56214,13.00868,77.5717
4,M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk,False,27-03-2023 08:46,28-03-2023 01:28,56,1001.867,13.00838,77.57156,2023-03-27 05:14:27+00:00,6.0,...,35,2023-03-27 04:45:26+00:00,2023-03-27 05:14:27+00:00,0.04,0,29.016667,13.00868,77.5717,13.00838,77.57156
5,M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk,False,27-03-2023 08:46,28-03-2023 01:28,56,1001.867,12.99925,77.57818,2023-03-27 05:49:23+00:00,11.0,...,64,2023-03-27 05:14:27+00:00,2023-03-27 05:49:23+00:00,1.24,34.933333,34.933333,13.00838,77.57156,12.99925,77.57818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91705,dnVidGo5c2QwcWFkOmJoNG5haDl0NXRlNGs=,True,27-03-2023 06:15,27-03-2023 21:44,28,928.150,12.97194,77.59371,2023-03-27 11:44:31+00:00,23.0,...,110,2023-03-27 09:48:18+00:00,2023-03-27 11:44:31+00:00,13.51,116.216667,116.216667,12.98961,77.71705,12.97194,77.59371
91706,dnVidGo5c2QwcWFkOmJoNG5haDl0NXRlNGs=,False,27-03-2023 06:15,27-03-2023 21:44,28,928.150,12.98974,77.71689,2023-03-27 13:42:14+00:00,11.0,...,82,2023-03-27 11:44:31+00:00,2023-03-27 13:42:14+00:00,13.49,117.716667,117.716667,12.97194,77.59371,12.98974,77.71689
91707,dnVidGo5c2QwcWFkOmJoNG5haDl0NXRlNGs=,False,27-03-2023 06:15,27-03-2023 21:44,28,928.150,12.98962,77.71693,2023-03-27 14:44:59+00:00,17.0,...,82,2023-03-27 13:42:14+00:00,2023-03-27 14:44:59+00:00,0.01,0,62.75,12.98974,77.71689,12.98962,77.71693
91708,dnVidGo5c2QwcWFkOmJoNG5haDl0NXRlNGs=,False,27-03-2023 06:15,27-03-2023 21:44,28,928.150,12.98960,77.71695,2023-03-27 15:33:19+00:00,30.0,...,82,2023-03-27 14:44:59+00:00,2023-03-27 15:33:19+00:00,0.0,0,48.333333,12.98962,77.71693,12.9896,77.71695


## Speed calculation

In [338]:
#motorised >=4km/hr, distance >= 1 km
#non motorized <= 4km /hr ,  distance < 1 km


In [339]:
deduplicated_data_new['speed'] = None
deduplicated_data_new['mode'] = None

In [340]:
import numpy as np

deduplicated_data_new['speed'] = deduplicated_data_new.apply(
    lambda row: (row['travel_distance'] / row['travel_time'] * 60)
                if not pd.isnull(row['travel_distance']) and not pd.isnull(row['travel_time']) and row['travel_time'] > 0
                else 0,
    axis=1)

# Determine mode based on speed and distance
deduplicated_data_new['mode'] = deduplicated_data_new.apply(
    lambda row: 'motorized' if row['speed'] >= 4 
                else ('non-motorized' if row['speed'] < 4 and not pd.isnull(row['travel_distance']) and row['travel_distance'] < 1 
                      else 'non-motorized'), 
    axis=1)


In [365]:
deduplicated_data_new.to_csv('final_Mode_removed.csv')

## Activity Mapping

In [371]:
# To test
# list_of_wards = []
# deduplicated_data.dropna(subset=['iso.date.time'], inplace=True)
# deduplicated_data['iso.date.time'] = pd.to_datetime(deduplicated_data['iso.date.time'])
# deduplicated_data['hour'] = deduplicated_data['iso.date.time'].dt.hour
# deduplicated_data['location_type'] = 'Temp'
# for index, rows in deduplicated_data.iterrows():
#     if rows['KGISWardNo'] not in list_of_wards and rows['KGISWardNo']==127:
#         list_of_wards.append(rows['KGISWardNo'])
# list_of_wards_type = []
# count_commercial = 0
# count_residency = 0
# for wards in list_of_wards:
#     print("WARD", wards)
#     count_commercial = 0
#     count_residency = 0
#     for index, rows in deduplicated_data.iterrows():
#         if wards == rows['KGISWardNo'] and wards == 127 and rows['hour'] > 8 and rows['hour'] < 20:
#             print(rows['hour'], rows['iso.date.time'])
#             count_commercial+=1
#         elif wards == rows['KGISWardNo'] and wards == 127 and (rows['hour'] >= 20 or rows['hour'] <= 8): 
#             print(rows['hour'], rows['iso.date.time'])
#             count_residency+=1
#     print(count_commercial, count_residency)
#     if count_commercial < count_residency:
#         list_of_wards_type.append('Residential')
#     else:
#         list_of_wards_type.append('Commerical')
#     print("Done")
        

# for index, rows in deduplicated_data.iterrows():
#     if rows['KGISWardNo'] == 127:
#         index_ward = list_of_wards.index(rows['KGISWardNo'])
#         deduplicated_data.at[index, 'location_type'] = list_of_wards_type[index_ward]





        



        

In [ ]:
list_of_wards = []
deduplicated_data.dropna(subset=['iso.date.time'], inplace=True)
deduplicated_data['iso.date.time'] = pd.to_datetime(deduplicated_data['iso.date.time'])
deduplicated_data['hour'] = deduplicated_data['iso.date.time'].dt.hour
deduplicated_data['location_type'] = 'Temp'
for index, rows in deduplicated_data.iterrows():
    if rows['KGISWardNo'] not in list_of_wards:
        list_of_wards.append(rows['KGISWardNo'])
list_of_wards_type = []
count_commercial = 0
count_residency = 0
for wards in list_of_wards:
    print("WARD", wards)
    count_commercial = 0
    count_residency = 0
    for index, rows in deduplicated_data.iterrows():
        if wards == rows['KGISWardNo'] and rows['hour'] > 8 and rows['hour'] < 20:
            print(rows['hour'], rows['iso.date.time'])
            count_commercial+=1
        elif wards == rows['KGISWardNo'] and (rows['hour'] >= 20 or rows['hour'] <= 8): 
            print(rows['hour'], rows['iso.date.time'])
            count_residency+=1
    print(count_commercial, count_residency)
    if count_commercial < count_residency:
        list_of_wards_type.append('Residential')
    else:
        list_of_wards_type.append('Commerical')
    print("Done")
        

for index, rows in deduplicated_data.iterrows():
    index_ward = list_of_wards.index(rows['KGISWardNo'])
    deduplicated_data.at[index, 'location_type'] = list_of_wards_type[index_ward]





        



        

In [370]:
# to test
for index, rows in deduplicated_data.iterrows():
    if rows['KGISWardNo'] == 127:
        print(rows['location_type'])

Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical
Commerical

In [320]:
# Drop rows with missing timestamps
deduplicated_data.dropna(subset=['iso.date.time'], inplace=True)

# Convert iso.date.time column to datetime
deduplicated_data['iso.date.time'] = pd.to_datetime(deduplicated_data['iso.date.time'])

# Extract hour from the datetime
deduplicated_data['hour'] = deduplicated_data['iso.date.time'].dt.hour

# Create a new column to identify residential or commercial
deduplicated_data['location_type'] = 'Commercial'  # Assume all locations are commercial by default

# Group by KGISWardNo and count events in different time intervals
grouped_data = deduplicated_data.groupby(['KGISWardNo'])
print(len(deduplicated_data), len(grouped_data))
# Function to determine location type
def get_location_type(group):
    night_events = group[(group['hour'] >= 0) & (group['hour'] < 8) | (group['hour'] >= 20) & (group['hour'] < 24)]
    day_events = group[(group['hour'] >= 8) & (group['hour'] < 20)]
    if len(night_events) > len(day_events):
        return 'Residential'
    else:
        return 'Commercial'

# Apply the function to each group and update the location_type column
deduplicated_data['location_type'] = grouped_data.apply(get_location_type)

# Drop the temporary columns
deduplicated_data.drop(columns=['hour'], inplace=True)

                                  device.id  andr.iph       first.event  \
0      M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk     False  27-03-2023 08:46   
1      M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk      True  27-03-2023 08:46   
2      M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk      True  27-03-2023 08:46   
3      M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk     False  27-03-2023 08:46   
4      M20wdjVuNDFlZmJwMDo5dnA1bDZrbG5pZnJk     False  27-03-2023 08:46   
...                                     ...       ...               ...   
18021  N25vdm80NW1oOTQxbTpmN2hwb3B2a3FnNWsy     False  27-03-2023 06:51   
18022  N25vdm80NW1oOTQxbTpmN2hwb3B2a3FnNWsy     False  27-03-2023 06:51   
18023  N25vdm80NW1oOTQxbTpmN2hwb3B2a3FnNWsy     False  27-03-2023 06:51   
18024  N25vdm80NW1oOTQxbTpmN2hwb3B2a3FnNWsy     False  27-03-2023 06:51   
19215  N2ZtMzZ2MHMybDQxZDo3cWpuMTVrY2VpdWpq      True  27-03-2023 12:34   

             last.event  tot.events  tot.event.dur.mins       lat      long  \
0      28-03-2023 01

In [312]:
# deduplicated_data.to_csv('Final.csv')